In [ ]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define CNN model
def create_cnn_model():
    model = Sequential()

    # Convolutional layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the layers
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Adding dropout to reduce overfitting

    # Output layer
    model.add(Dense(4, activation='softmax'))  # 4 classes (Mild, Moderate, Non, Very_Mild)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Create an instance of the model
model = create_cnn_model()

# Print the model summary
model.summary()

# Create ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load training and testing datasets
train_data = train_datagen.flow_from_directory(
    'path_to_training_data',  # Replace with your dataset path
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    'path_to_test_data',  # Replace with your dataset path
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Train the model
history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_data=test_data,
    validation_steps=test_data.samples // test_data.batch_size,
    epochs=10  # Adjust based on your needs
)

# Save the model
model.save('cnn_alzheimer_classification.h5')


In [9]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the dataset
train_dataset = load_dataset('Falah/Alzheimer_MRI', split='train')
test_dataset = load_dataset('Falah/Alzheimer_MRI', split='test')


def preprocess_data(example):
    # Convert the image to a TensorFlow tensor
    image = example['image']
    image = tf.convert_to_tensor(image, dtype=tf.float32)  # Ensure it's a float32 tensor
    
    # Add a channel dimension if missing (grayscale to RGB conversion)
    if len(image.shape) == 2:  # If shape is (128, 128), we add a channel dimension
        image = tf.expand_dims(image, axis=-1)  # Add a channel dimension, becomes (128, 128, 1)
    
    # Convert grayscale to RGB by repeating the single channel 3 times
    image = tf.image.grayscale_to_rgb(image)  # Now the shape will be (128, 128, 3)
    
    # Normalize the image to range [0, 1]
    image = image / 255.0
    
    # Ensure image has shape (128, 128, 3)
    image = tf.ensure_shape(image, (128, 128, 3))
    
    # Convert label to one-hot encoding (4 classes)
    label = tf.one_hot(example['label'], depth=4)
    
    # Return a dictionary with the image and label
    return {'image': image, 'label': label}

def create_cnn_model():
    model = Sequential()

    # Convolutional layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the layers
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout to reduce overfitting

    # Output layer
    model.add(Dense(4, activation='softmax'))  # 4 classes

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Apply preprocessing to the datasets
train_dataset = train_dataset.map(preprocess_data)
test_dataset = test_dataset.map(preprocess_data)

# Function to generate data for tf.data.Dataset
def generator(dataset):
    for example in dataset:
        yield example['image'], example['label']

# Convert Hugging Face Dataset to tf.data.Dataset
BATCH_SIZE = 32

train_data = tf.data.Dataset.from_generator(
    lambda: generator(train_dataset),
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(4,), dtype=tf.float32)
    )
)

test_data = tf.data.Dataset.from_generator(
    lambda: generator(test_dataset),
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(4,), dtype=tf.float32)
    )
)

# Batch, shuffle, and prefetch the datasets
train_data = train_data.batch(BATCH_SIZE).shuffle(1024).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

# Check batch shapes for debugging
for images, labels in train_data.take(1):
    print(f"Image Batch Shape: {images.shape}")  # Should be (batch_size, 128, 128, 3)
    print(f"Label Batch Shape: {labels.shape}")  # Should be (batch_size, 4)
    
# Create an instance of the model
model = create_cnn_model()

# Print the model summary
model.summary()

# Train the model
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10  # Adjust based on your needs
)

# Save the model
model.save("alzheimer_cnn_model.keras")


2024-10-15 00:38:42.738097: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 11 of 1024
2024-10-15 00:38:52.749389: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 22 of 1024
2024-10-15 00:39:03.665698: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 34 of 1024
2024-10-15 00:39:23.411969: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 56 of 1024
2024-10-15 00:39:42.835771: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 77 of 1024
2024-10-15 00:39:53.558266: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while

Image Batch Shape: (32, 128, 128, 3)
Label Batch Shape: (32, 4)


2024-10-15 00:40:57.666045: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
2024-10-15 00:40:57.683832: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

160/160 ━━━━━━━━━━━━━━━━━━━━ 202s 369ms/step - accuracy: 0.8785 - loss: 0.2898 - val_accuracy: 0.9187 - val_loss: 0.2355
Epoch 10/10


2024-10-15 01:11:48.691760: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 11 of 1024
2024-10-15 01:12:09.353785: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 34 of 1024
2024-10-15 01:12:29.247705: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 56 of 1024
2024-10-15 01:12:49.348947: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 78 of 1024
2024-10-15 01:13:08.706231: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 99 of 1024
2024-10-15 01:13:29.459127: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while

  1/160 ━━━━━━━━━━━━━━━━━━━━ 6:25:27 145s/step - accuracy: 0.9062 - loss: 0.1930

2024-10-15 01:14:03.860803: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


160/160 ━━━━━━━━━━━━━━━━━━━━ 204s 367ms/step - accuracy: 0.9154 - loss: 0.2100 - val_accuracy: 0.9312 - val_loss: 0.2285
